# Pastas Projects using Arctic<a id="top"></a>

[Arctic](https://arctic.readthedocs.io/en/latest/) is a timeseries / dataframe database that sits atop [MongoDB](https://www.mongodb.com). This notebook shows how [Pastas](https://pastas.readthedocs.io/en/latest/) timeseries and models can be managed and stored in a MongoDB. 

## Content
1. [Getting started](#1)
2. [The ArcticPastas object](#2)
  1. [Connect to a database](#2.1)
  2. [Structure](#2.2)
3. [Managing timeseries](#3)
  1. [Adding oseries and stresses](#3.1)
  2. [Accessing timeseries and metadata](#3.2)
  3. [Overview of oseries and stresses](#3.3)
4. [Managing Pastas models](#4)
  1. [Creating a model](#4.1)
  2. [Storing a model](#4.2)
  3. [Loading a model](#4.3)
  4. [Overview of models](#4.4)
5. [Bulk operations](#5)

<hr>

## [1. Getting started](#top)<a id="1"></a>

Use the following steps to get your PC ready for this notebook:

1. Install [MongoDB community](https://www.mongodb.com/download-center/community).
2. Install Arctic using pip: `pip install arctic`.
3. Run MongoDB by typing `mongod --dbpath <your_path_here>` in your terminal, or if on Windows use the batch script `start_mongodb.bat`.
4. If no errors were encountered, you're all set!

<hr>

## [2. The ArcticPastas object](#top)<a id="2"></a>
This sections shows how to initialize an `ArcticPastas` object to create a new database or connect to an existing one.

Import `ArcticPastas` and some other modules:

In [1]:
import pastas as ps

import sys
sys.path.insert(1, "../..")

from pastas_projects import ArcticPastas

C:\Users\dbrak\Anaconda3\lib\site-packages\arctic\store\_pandas_ndarray_store.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import DataFrame, Series, Panel


### [2.1 Connect to a database](#top)<a id="2.1"></a>

Provide information about the database. The connection string tells Arctic where the database is running (by default, if running locally the address is `localhost:<port number>`. The project name is the user specified name for the database. If the database already exists, Arctic will connect to that existing database.

In [2]:
connstr = "mongodb://localhost:27017/"  # for docker container with name 'mongodb' running mongodb
projectname = "aaenmaas"

Initialize an ArcticPastas object. This is an object that connects to an existing database or initializes a new database if `projectname` does not yet exist.

In [3]:
pr = ArcticPastas(connstr, projectname)

Let's take a look at `pr`. This shows us how many oseries, stresses and models are contained in the database:

In [4]:
pr

<ArcticPastas object> 'aaenmaas': 481 oseries, 170 stresses, 481 models

### [2.2 Structure](#top)<a id="2.2"></a>

The database for this project contains 3 so-called libraries. Each of these contains specific data related to the project. The three libraries are:
- oseries
- stresses
- models

These libraries can be accessed through `pr.get_library()` or through the attributes `pr.lib_<libname>`:

In [6]:
pr.get_library("oseries")

    <ArcticLibrary at 0x29a6b45e908, arctic_aaenmaas.oseries>
        <Arctic at 0x29a71524d68, connected to MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, maxpoolsize=4, sockettimeoutms=600000, connecttimeoutms=2000, serverselectiontimeoutms=30000)>

In [7]:
pr.lib_oseries

    <ArcticLibrary at 0x29a6b45e908, arctic_aaenmaas.oseries>
        <Arctic at 0x29a71524d68, connected to MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, maxpoolsize=4, sockettimeoutms=600000, connecttimeoutms=2000, serverselectiontimeoutms=30000)>

These libraries contain the timeseries and the models. Within these libraries stored objects are labelled with so-called symbols. These symbols are used to read and write the data. 

<hr>

## [3. Managing timeseries](#top)<a id="3"></a>

This section explains how timeseries can be added or loaded from the database

### [3.1 Adding oseries and stresses](#top)<a id="3.1"></a>

Since this notebook is using an existing database, I'm using an existing timeseries to illustrate the adding of new timeseries. More on the accessing of existing timeseries in the next section.

In [8]:
ts = pr.get_oseries("WIJB020_G")
ts.head()

,value,flag,comment,flagSource,user
index,,,,,
2012-06-19 11:00:00,7.338,0,,,
2012-06-19 12:00:00,7.331,0,,,
2012-06-19 13:00:00,7.330,0,,,
2012-06-19 14:00:00,7.328,0,,,
2012-06-19 15:00:00,7.324,0,,,


Adding oseries or stresses is possible through `pr.add_oseries()` or `pr.add_stresses()`. Metadata can be optionally be provided as a dictionary. The example shows how an oseries can be added, the recipe is identical for `pr.add_stresses()`. The only thing to keep in mind when adding stresses is to add a `kind` attribute to the metadata dictionary so that different types of stresses (i.e. precipitation or evaporation) can be distinguished.

In [8]:
pr.add_oseries(ts, "test2", metadata={"in_calibration_dataset": True})

An oseries can be deleted with `pr.del_oseries()`:

In [9]:
pr.del_oseries("test2")

### [3.2 Accessing timeseries and metadata](#top)<a id="3.2"></a>

Timeseries can be accessed through `pr.get_oseries()` or `pr.get_stresses()`. These methods accept single symbols or lists of symbols. In the latter case a list of dataframes is returned.

In [10]:
ts = pr.get_oseries(["WIJB020_G", "STRA001_G"])
ts

{'WIJB020_G':                      value  flag comment flagSource user
 index                                                   
 2012-06-19 11:00:00  7.338     0                        
 2012-06-19 12:00:00  7.331     0                        
 2012-06-19 13:00:00  7.330     0                        
 2012-06-19 14:00:00  7.328     0                        
 2012-06-19 15:00:00  7.324     0                        
 ...                    ...   ...     ...        ...  ...
 2019-05-27 08:00:00  6.811     0                        
 2019-05-27 09:00:00  6.810     0                        
 2019-05-27 10:00:00  6.807     0                        
 2019-05-27 11:00:00  6.806     0                        
 2019-05-27 12:00:00  6.803     0                        
 
 [57524 rows x 5 columns],
 'STRA001_G':                       value  flag comment flagSource user
 index                                                    
 2010-03-25 11:00:00  20.586     0                        
 2010-03-25 12

The precipitation and evaporation data is stored by its station number (perhaps a more convenient symbol can be applied in the future).

In [11]:
stresses = pr.get_stresses(['10', '980'])
stresses

{'10':                   RD
 YYYYMMDD            
 1906-01-01  0.000000
 1906-01-02  0.000000
 1906-01-03  0.000000
 1906-01-04  0.001218
 1906-01-05  0.005988
 ...              ...
 2019-09-16  0.000000
 2019-09-17  0.000100
 2019-09-18  0.001200
 2019-09-19  0.000200
 2019-09-20  0.000000
 
 [41536 rows x 1 columns], '980':                   RD
 YYYYMMDD            
 1951-01-01  0.000000
 1951-01-02  0.000000
 1951-01-03  0.000000
 1951-01-04  0.003951
 1951-01-05  0.002029
 ...              ...
 2019-09-06  0.000792
 2019-09-07  0.000000
 2019-09-08  0.008034
 2019-09-09  0.000000
 2019-09-10  0.000000
 
 [25090 rows x 1 columns]}

The metadata of a timeseries can be accessed through `pr.get_metadata()`. Provide the library name and the symbol to load the metadata for an oseries or a stress.

In [12]:
meta = pr.get_metadata('oseries', "WIJB020_G")
meta

{'type': 'instantaneous',
 'locationId': 'WIJB020_G',
 'parameterId': 'GW.meting.totaalcorrectie',
 'timeStep': {'unit': 'nonequidistant'},
 'startDate': {'date': '2012-06-19', 'time': '11:00:00'},
 'endDate': {'date': '2019-05-27', 'time': '12:00:00'},
 'missVal': 'NaN',
 'longName': None,
 'x': '161634.0',
 'y': '404649.0',
 'units': 'mNAP',
 'sourceOrganisation': 'Artesia',
 'sourceSystem': 'art_diver03.582b',
 'fileDescription': None,
 'region': None,
 '_updated': '2019-09-12 13:01:48.204106',
 'datastore': '..\\traval\\extracted_data\\pystore\\aaenmaas',
 'name': 'WIJB020_G'}

### [3.3 Overview of oseries and stresses](#top)<a id="3.3"></a>

An overview of the oseries and stresses is available through `pr.oseries` and `pr.stresses`. These are dataframes containing the metadata of all the timeseries. These dataframes are cached for performance. The cache is cleared when a timeseries is added or modified in the database. 

In [13]:
pr.oseries.head()

,type,locationId,parameterId,timeStep,startDate,endDate,missVal,longName,x,y,units,sourceOrganisation,sourceSystem,fileDescription,region,_updated,datastore,geometry
name,,,,,,,,,,,,,,,,,,
103JVM_boven_O,instantaneous,103JVM_boven_O,GW.meting.totaalcorrectie,{'unit': 'nonequidistant'},"{'date': '2015-09-10', 'time': '18:00:00'}","{'date': '2019-06-21', 'time': '08:00:00'}",NaN,None,162831.0,417973.0,mNAP,Artesia,art_diver03.582b,None,None,2019-09-12 13:01:27.745137,..\traval\extracted_data\pystore\aaenmaas,POINT (162831 417973)
103KBC_beneden_O,instantaneous,103KBC_beneden_O,GW.meting.totaalcorrectie,{'unit': 'nonequidistant'},"{'date': '2015-09-10', 'time': '18:00:00'}","{'date': '2019-06-21', 'time': '09:00:00'}",NaN,None,166688.0,416940.0,mNAP,Artesia,art_diver03.582b,None,None,2019-09-12 13:01:28.120103,..\traval\extracted_data\pystore\aaenmaas,POINT (166688 416940)
103SZB_beneden_O,instantaneous,103SZB_beneden_O,GW.meting.totaalcorrectie,{'unit': 'nonequidistant'},"{'date': '2015-09-10', 'time': '18:00:00'}","{'date': '2019-06-21', 'time': '08:00:00'}",NaN,None,164713.0,418134.0,mNAP,Artesia,art_diver03.582b,None,None,2019-09-12 13:01:28.455111,..\traval\extracted_data\pystore\aaenmaas,POINT (164713 418134)
114DZS_O,instantaneous,114DZS_O,GW.meting.totaalcorrectie,{'unit': 'nonequidistant'},"{'date': '2015-02-23', 'time': '16:00:00'}","{'date': '2015-12-18', 'time': '09:00:00'}",NaN,None,190012.0,402860.0,mNAP,Artesia,art_diver03.582b,None,None,2019-09-12 13:01:28.756105,..\traval\extracted_data\pystore\aaenmaas,POINT (190012 402860)
253GC_boven_O,instantaneous,253GC_boven_O,GW.meting.totaalcorrectie,{'unit': 'nonequidistant'},"{'date': '2015-05-25', 'time': '12:33:07'}","{'date': '2018-02-21', 'time': '17:00:00'}",NaN,None,179254.0,391482.0,mNAP,Artesia,art_diver03.582b,None,None,2019-09-12 13:01:29.099107,..\traval\extracted_data\pystore\aaenmaas,POINT (179254 391482)


In [16]:
pr.stresses.head()

,x,y,station,group,_updated,datastore,kind,geometry
name,,,,,,,,
RD Hollum,171328.112799,605131.163945,10,knmi_data,2019-10-04 16:04:46.908179,C:\GitHub\traval\extracted_data\pystore\knmi,prec,POINT (171328.112798747 605131.1639452977)
RD West Terschelling,143620.063215,597697.877355,11,knmi_data,2019-10-04 16:04:46.603778,C:\GitHub\traval\extracted_data\pystore\knmi,prec,POINT (143620.0632145553 597697.8773549523)
RD Schiermonnikoog,207837.724003,609105.113327,12,knmi_data,2019-10-04 16:04:45.908447,C:\GitHub\traval\extracted_data\pystore\knmi,prec,POINT (207837.7240030442 609105.1133266287)
RD Oost Vlieland,133594.727540,588458.663083,15,knmi_data,2019-10-04 16:04:46.239686,C:\GitHub\traval\extracted_data\pystore\knmi,prec,POINT (133594.7275398257 588458.6630831728)
RD Nes (Ameland),180188.996497,605169.642060,18,knmi_data,2019-10-04 16:04:46.659799,C:\GitHub\traval\extracted_data\pystore\knmi,prec,POINT (180188.9964972839 605169.6420600282)


<hr>

## [4. Managing Pastas models](#top)<a id="4"></a>

This section shows how Pastas models can be created, stored, and loaded from the database.

### [4.1 Creating a model](#top)<a id="4.1"></a>
Creating a new model is straightforward using `pr.create_model()`. The `add_recharge` keyword argument allows the user to choose (default is True) whether recharge is automatically added to the model using the closest precipitation and evaporation stations in the stresses library.

In [17]:
ml = pr.create_model("WIJB020_G", add_recharge=True)
ml

INFO: Cannot determine frequency of series WIJB020_G
INFO:pastas.timeseries:Cannot determine frequency of series WIJB020_G
INFO: Time Series WIJB020_G: 2 nan-value(s) was/were found and filled with: drop
INFO:pastas.timeseries:Time Series WIJB020_G: 2 nan-value(s) was/were found and filled with: drop
INFO: Inferred frequency from time series RD Dinther: freq=D 
INFO:pastas.timeseries:Inferred frequency from time series RD Dinther: freq=D 
INFO: Inferred frequency from time series EV24 Volkel: freq=D 
INFO:pastas.timeseries:Inferred frequency from time series EV24 Volkel: freq=D 


Model(oseries=WIJB020_G, name=WIJB020_G, constant=True, noisemodel=True)

### [4.2 Storing a model](#top)<a id="4.2"></a>
The model that was created in the previous step is not automatically stored in the models library. Use `pr.add_model()` to store the model. If the model already exists, an Exception is raised warning the use the model is already in the library. Use `add_version=True` to add the model anyway. This creates a new version of the object in the library.

**Note:**
The model is stored without the timeseries. It is assumed the timeseries are already stored in the oseries or stresses libraries, making it redundant to store these again in most cases. Obviously this has the potential downside that modifications to a timeseries prior to using it in a model will not be saved. In this implementation, the user is expected to add a new timeseries under a new name or version to the oseries and stresses libraries and create a new model using that data.

In [16]:
pr.add_model(ml, add_version=True)

### [4.3 Loading a model](#top)<a id="4.3"></a>

Loading a stored model is simple using `pr.get_models()`.

The model is stored as a dictionary (see `ml.to_dict()`) without the timeseries data. The timeseries in the model are picked up based on the names of those series from the respective libraries (oseries or stresses).

In [17]:
ml2 = pr.get_models(["WIJB020_G"])
ml2

INFO: Cannot determine frequency of series WIJB020_G
INFO: Time Series WIJB020_G: 2 nan-value(s) was/were found and filled with: drop


Model(oseries=WIJB020_G, name=WIJB020_G, constant=True, noisemodel=True)

### [4.4 Overview of models](#top)<a id="4.4"></a>

An overview of the models is available through `pr.models` which lists the names of all the models:

In [9]:
pr.models[:10]

['103JVM_boven_O',
 '103KBC_beneden_O',
 '103SZB_beneden_O',
 '114DZS_O',
 '253GC_boven_O',
 '253GG_boven_O',
 '253LCG_boven_O',
 '253LC_boven_O',
 '253M_boven_O',
 '275HGB_boven_O']

<hr>

## [5. Bulk operations](#top)<a id="5"></a>

The following bulk operations are available:
- `create_models`: create models for all oseries in database
- `solve_models`: solve all or selection of models in database
- `model_results`: get results for all or selection of models in database

The `pr.create_models()` method allows the user to get models for all or a selection of oseries in the database. Options include:
- selecting specific oseries to create models for
- automatically adding recharge based on nearest precipitation and evaporation stresses
- solving the models
- storing the models in the models library

**Note**: when using the progressbar (default), for a prettier result the pastas log level should be set to ERROR using: `ps.set_log_level("ERROR")`.

In [5]:
ps.set_log_level("ERROR")

In [6]:
# mls = pr.create_models(oseries=["WIJB020_G", "STRA001_G"], store=False)
# mls
mls = pr.create_models(store=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 481/481 [01:29<00:00,  5.38it/s]


To solve all or a selection of models use `pr.solve_models()`. Options for this method include:
- selecting models to solve
- store results in models library
- raise error (or not) when solving fails
- print solve reports

In [7]:
# pr.solve_models(mls=["WIJB020_G", "STRA001_G"], store_result=True)
pr.solve_models(store_result=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 481/481 [07:15<00:00,  1.11it/s]


Obtaining the model results (parameters, EVP and some other statistics) requires the art_tools module. Results can be obtained for all or a selection of models. The result is a DataFrame with the results:

In [26]:
# results = pr.model_results(mls=["WIJB020_G", "STRA001_G"])
# results.T
results = pr.model_results()
results.head()

flopy is installed in C:\Users\dbrak\Anaconda3\lib\site-packages\flopy


100%|████████████████████████████████████████████████████████████████████████████████| 481/481 [02:13<00:00,  3.60it/s]


,recharge_A,recharge_n,recharge_a,recharge_f,constant_d,noise_alpha,recharge_A_stderr,recharge_n_stderr,recharge_a_stderr,recharge_f_stderr,constant_d_stderr,noise_alpha_stderr,evp,number of observations used in calibration,memory recharge [days],rfunc recharge,calibration period [days]
103JVM_boven_O,277.677,1.16774,115.153,-1.07852,5.20276,92.146,16.0836,0.00910458,6.68289,0.0495698,0.0255696,6.64933,69.244,NaN,297.995,Gamma,1380
103KBC_beneden_O,381.306,1.04425,116.89,-0.636491,9.1122,30.0863,17.5488,0.00710612,6.39225,0.0274758,0.0276627,1.34556,66.7966,NaN,278.074,Gamma,1380
103SZB_beneden_O,205.948,1.12655,84.871,-0.822262,5.70939,97.9455,11.2567,0.00649306,4.50049,0.0352531,0.0181701,7.66537,69.1101,NaN,213.771,Gamma,1380
114DZS_O,1.839,10,0.140495,-0.958413,16.3206,22.1831,0.140127,1.34751,0.0212806,0.292327,0.00701472,1.92568,0,NaN,1.99587,Gamma,298
253GC_boven_O,96.303,0.799693,147.693,-0.173348,18.9622,13.8294,9.19931,0.00886058,20.403,0.0321927,0.0191603,0.476907,39.0561,NaN,287.218,Gamma,1003
